In [ ]:
from collections import defaultdict
import heapq
import math
import sys
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
MAX_ITER = 1000

In [ ]:
def isGoal(state, currentState):
    if (len(state)== len(currentState[0])+1 and state[0] == state[len(state)-1]):
        return True
    return False

In [ ]:
def plot(iteration, heuristic, state, optimum):
    """
    Plots the resulting heuristic on each iteration.
    :param iteration: Total number of iterations
    :param heuristic: List of heuristic score
    :param state: The final state of queens
    :param optimum: The highest possible score
    :return:
    """
    # For results which break the loop in the algorithm
    if iteration == MAX_ITER:
        heuristic.append(g(state))

    plt.plot(range(iteration ), heuristic, 'teal')
    plt.plot(iteration, optimum, 'cyan')

    plt.xlabel("Iterations")
    plt.ylabel("COST")
    plt.show()

In [ ]:
def getState(state):
    n = int(input("Enter no. of cities: "))
    print("Input the city matrix")
    state = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i+1,n):
            state[i][j] = state[j][i] = int(input(f"Enter dist between city {i} and {j}: "))
    return state

In [ ]:
def print_sol(state, tsp):
    if isGoal(state, tsp): print('\nSolved Puzzle!')
    print('\nFinal State:')
    print(state)
    return

In [ ]:
def expand(state, n):
    neighbours = []
    for i in range(n):
        state_copy = state.copy()
        if i not in state or (n == len(state) and i == state[0]):
            state_copy.append(i)
            neighbours.append(state_copy)

    return neighbours

In [ ]:
def f(state, tsp):
    # lower bound h(n) + g(n)
    n = len(tsp)
    value = g(state, tsp)
    # if n==1: return 0
    for i in range(n):
        curr = tsp[i].copy()
        curr.sort()
        if i in state and (i != state[0] and i != state[len(state)-1]):
            continue
        elif i in state:
            value+=curr[0]
        else:
            value+=(curr[0]+curr[1])
    return value

In [ ]:
def g(state, tsp):
    # actual path cost
    n = len(state)
    cost=0
    if n==1: return 0
    for i in range(1,n):
        cost += tsp[state[i-1]][state[i]]     
    return cost    

In [ ]:

def random_restart(noOfCities):
    cities = list(range(1,noOfCities))
    solution = [0]
    for _ in range(1,noOfCities):
        randomCity = cities[random.randint(0, len(cities) - 1)]
        solution.append(randomCity)
        cities.remove(randomCity)

    solution.append(0)
    return solution

In [ ]:
def getNeighbours(state):
    neighbours = []
    for i in range(1,len(state)-1):
        for j in range(i+1,len(state)-1):
            neighbour = state.copy()
            neighbour[i],neighbour[j] = state[j],state[i]
            neighbours.append(neighbour)
    return neighbours


In [ ]:
def UCS(tsp):
    optimum = i = 0
    evaluation = []
    pq = [(0,[0])]
    heapq.heapify(pq)
    cur_v = []
    while(len(pq)>0):
        i+=1
        v, cur_v = heapq.heappop(pq)
        print(f"Iteration {i}: Evaluation = {g(cur_v, tsp)} state = {cur_v}")
        evaluation.append(g(cur_v, tsp))
        if isGoal(cur_v, tsp) or i == MAX_ITER:
            break

        neighbour = expand(cur_v, len(tsp))
        for v in neighbour:
            heapq.heappush(pq, [g(v, tsp), list(v)])

    print_sol(cur_v, tsp,)
    # plot(i, evaluation, cur_v, optimum)
    return


In [ ]:
def Astar(state):
    optimum = i = 0
    evaluation = []
    pq = [(0,[0])]
    heapq.heapify(pq)
    cur_v = []
    while(len(pq)>0):
        v, cur_v = heapq.heappop(pq)
        i += 1
        print(f"Iteration {i}: Evaluation = {f(cur_v, state)} state = {cur_v}")
        evaluation.append(f(cur_v, state))
        if isGoal(cur_v, state) or i==MAX_ITER:
            break

        neighbour = expand(cur_v, len(state))
        for v in neighbour:
            heapq.heappush(pq, [f(v, state), list(v)])

    print_sol(cur_v, state)
    plot(i, evaluation, cur_v, optimum)
    return

In [ ]:

def depthFirstBnB(tsp):
    optimum = i = 0
    evaluation = []
    stack = [[0]]
    bound = 1e5
    while(len(stack)>0):
        cur_v = stack.pop()
        i += 1
        print(f"Iteration {i}: Evaluation = {f(cur_v, tsp)} state = {cur_v}")
        evaluation.append(f(cur_v, tsp))
        if isGoal(cur_v, tsp) and bound > f(cur_v, tsp):
            bound = f(cur_v, tsp)
            best_sol = cur_v
        
        if f(cur_v, tsp) < bound:
            neighbour = expand(cur_v, len(tsp))
            for v in neighbour:
                if(f(v, tsp) < bound): 
                    stack.append(list(v))

    print_sol(best_sol, tsp)
    plot(i, evaluation, best_sol, optimum)
    return

In [ ]:
def localHillClimbing(tsp):
    optimum = i = 0
    n = len(tsp)
    cur_v = random_restart(n)
    next_v = getNeighbours(cur_v)
    next_best = 0
    for v in next_v:
        if next_best < f(v, tsp):
            next_best = f(v, tsp)
            next_best_v = v
    evaluation = []
    i += 1
    print(f"Iteration {i}: Evaluation = {f(cur_v, tsp)} state = {cur_v}")
    evaluation.append(f(cur_v, tsp))
    while(next_best < f(cur_v, tsp) and i<MAX_ITER):
        i += 1
        print(f"Iteration {i}: Evaluation = {f(cur_v, tsp)} state = {cur_v}")
        evaluation.append(f(cur_v, tsp))
        cur_v = next_best_v
        next_v = getNeighbours(cur_v)
        next_best = 0
        for v in next_v:
            if next_best < f(v, tsp):
                next_best = f(v, tsp)
                next_best_v = v
    print_sol(cur_v, tsp)
    plot(i, evaluation, cur_v, optimum)
    return

In [ ]:
    choice=0
    initialState = [[]]
    initialState = getState(initialState)
    while(choice!=5):
        choice = int(input('1. UCS\n2. A*\n3. Depth-first branch and bound\n4. Hill Climbing\n5. Exit\n'))
        if choice==1: UCS(initialState)
        elif(choice==2): Astar(initialState)
        elif choice==3: depthFirstBnB(initialState)
        elif choice==4: localHillClimbing(initialState)
        elif choice==5: break